190803 조진욱

3250 feats-> 532feats using shap[LB : 0.436]

https://www.kaggle.com/youhanlee/3250feats-532-feats-using-shap-lb-0-436


# Preparation

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!mkdir -p ~/.kaggle
!cp /content/drive/My\ Dirve/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!pip install kaggle
!pip install shap

In [0]:
!unzip /contetn/train.csv.zip

# 필사시작


# 1. Check Datasets




1.1 import and read dataset
---



In [0]:
import seaborn as sns

sns.set(font_scale = 2.2)
plt.style.use('seaborn')

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, ShuffleSplit
from sklearn.metrics import f1_score

import itertools
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier

import shap
from tqdm import tqdm
import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

import time



1.2 Make description df
---



In [0]:
description = [
("v2a1"," Monthly rent payment"),
("hacdor"," =1 Overcrowding by bedrooms"),
("rooms","  number of all rooms in the house"),
("hacapo"," =1 Overcrowding by rooms"),
("v14a"," =1 has toilet in the household"),
("refrig"," =1 if the household has refrigerator"),
("v18q"," owns a tablet"),
("v18q1"," number of tablets household owns"),
("r4h1"," Males younger than 12 years of age"),
("r4h2"," Males 12 years of age and older"),
("r4h3"," Total males in the household"),
("r4m1"," Females younger than 12 years of age"),
("r4m2"," Females 12 years of age and older"),
("r4m3"," Total females in the household"),
("r4t1"," persons younger than 12 years of age"),
("r4t2"," persons 12 years of age and older"),
("r4t3"," Total persons in the household"),
("tamhog"," size of the household"),
("tamviv"," number of persons living in the household"),
("escolari"," years of schooling"),
("rez_esc"," Years behind in school"),
("hhsize"," household size"),
("paredblolad"," =1 if predominant material on the outside wall is block or brick"),
("paredzocalo"," =1 if predominant material on the outside wall is socket (wood, zinc or absbesto"),
("paredpreb"," =1 if predominant material on the outside wall is prefabricated or cement"),
("pareddes"," =1 if predominant material on the outside wall is waste material"),
("paredmad"," =1 if predominant material on the outside wall is wood"),
("paredzinc"," =1 if predominant material on the outside wall is zink"),
("paredfibras"," =1 if predominant material on the outside wall is natural fibers"),
("paredother"," =1 if predominant material on the outside wall is other"),
("pisomoscer"," =1 if predominant material on the floor is mosaic ceramic   terrazo"),
("pisocemento"," =1 if predominant material on the floor is cement"),
("pisoother"," =1 if predominant material on the floor is other"),
("pisonatur"," =1 if predominant material on the floor is  natural material"),
("pisonotiene"," =1 if no floor at the household"),
("pisomadera"," =1 if predominant material on the floor is wood"),
("techozinc"," =1 if predominant material on the roof is metal foil or zink"),
("techoentrepiso"," =1 if predominant material on the roof is fiber cement,   mezzanine "),
("techocane"," =1 if predominant material on the roof is natural fibers"),
("techootro"," =1 if predominant material on the roof is other"),
("cielorazo"," =1 if the house has ceiling"),
("abastaguadentro"," =1 if water provision inside the dwelling"),
("abastaguafuera"," =1 if water provision outside the dwelling"),
("abastaguano"," =1 if no water provision"),
("public"," =1 electricity from CNFL,  ICE, ESPH/JASEC"),
("planpri"," =1 electricity from private plant"),
("noelec"," =1 no electricity in the dwelling"),
("coopele"," =1 electricity from cooperative"),
("sanitario1"," =1 no toilet in the dwelling"),
("sanitario2"," =1 toilet connected to sewer or cesspool"),
("sanitario3"," =1 toilet connected to  septic tank"),
("sanitario5"," =1 toilet connected to black hole or letrine"),
("sanitario6"," =1 toilet connected to other system"),
("energcocinar1"," =1 no main source of energy used for cooking (no kitchen)"),
("energcocinar2"," =1 main source of energy used for cooking electricity"),
("energcocinar3"," =1 main source of energy used for cooking gas"),
("energcocinar4"," =1 main source of energy used for cooking wood charcoal"),
("elimbasu1"," =1 if rubbish disposal mainly by tanker truck"),
("elimbasu2"," =1 if rubbish disposal mainly by botan hollow or buried"),
("elimbasu3"," =1 if rubbish disposal mainly by burning"),
("elimbasu4"," =1 if rubbish disposal mainly by throwing in an unoccupied space"),
("elimbasu5"," =1 if rubbish disposal mainly by throwing in river,   creek or sea"),
("elimbasu6"," =1 if rubbish disposal mainly other"),
("epared1"," =1 if walls are bad"),
("epared2"," =1 if walls are regular"),
("epared3"," =1 if walls are good"),
("etecho1"," =1 if roof are bad"),
("etecho2"," =1 if roof are regular"),
("etecho3"," =1 if roof are good"),
("eviv1"," =1 if floor are bad"),
("eviv2"," =1 if floor are regular"),
("eviv3"," =1 if floor are good"),
("dis"," =1 if disable person"),
("male"," =1 if male"),
("female"," =1 if female"),
("estadocivil1"," =1 if less than 10 years old"),
("estadocivil2"," =1 if free or coupled uunion"),
("estadocivil3"," =1 if married"),
("estadocivil4"," =1 if divorced"),
("estadocivil5"," =1 if separated"),
("estadocivil6"," =1 if widow/er"),
("estadocivil7"," =1 if single"),
("parentesco1"," =1 if household head"),
("parentesco2"," =1 if spouse/partner"),
("parentesco3"," =1 if son/doughter"),
("parentesco4"," =1 if stepson/doughter"),
("parentesco5"," =1 if son/doughter in law"),
("parentesco6"," =1 if grandson/doughter"),
("parentesco7"," =1 if mother/father"),
("parentesco8"," =1 if father/mother in law"),
("parentesco9"," =1 if brother/sister"),
("parentesco10"," =1 if brother/sister in law"),
("parentesco11"," =1 if other family member"),
("parentesco12"," =1 if other non family member"),
("idhogar"," Household level identifier"),
("hogar_nin"," Number of children 0 to 19 in household"),
("hogar_adul"," Number of adults in household"),
("hogar_mayor"," # of individuals 65+ in the household"),
("hogar_total"," # of total individuals in the household"),
("dependency"," Dependency rate"),
("edjefe"," years of education of male head of household"),
("edjefa"," years of education of female head of household"),
("meaneduc","average years of education for adults (18+)"),
("instlevel1"," =1 no level of education"),
("instlevel2"," =1 incomplete primary"),
("instlevel3"," =1 complete primary"),
("instlevel4"," =1 incomplete academic secondary level"),
("instlevel5"," =1 complete academic secondary level"),
("instlevel6"," =1 incomplete technical secondary level"),
("instlevel7"," =1 complete technical secondary level"),
("instlevel8"," =1 undergraduate and higher education"),
("instlevel9"," =1 postgraduate higher education"),
("bedrooms"," number of bedrooms"),
("overcrowding"," # persons per room"),
("tipovivi1"," =1 own and fully paid house"),
("tipovivi2"," =1 own,   paying in installments"),
("tipovivi3"," =1 rented"),
("tipovivi4"," =1 precarious"),
("tipovivi5"," =1 other(assigned"),
("computer"," =1 if the household has notebook or desktop computer,   borrowed)"),
("television"," =1 if the household has TV"),
("mobilephone"," =1 if mobile phone"),
("qmobilephone"," # of mobile phones"),
("lugar1"," =1 region Central"),
("lugar2"," =1 region Chorotega"),
("lugar3"," =1 region PacÃƒÂ­fico central"),
("lugar4"," =1 region Brunca"),
("lugar5"," =1 region Huetar AtlÃƒÂ¡ntica"),
("lugar6"," =1 region Huetar Norte"),
("area1"," =1 zona urbana"),
("area2"," =2 zona rural"),
("age"," Age in years"),
("SQBescolari"," escolari squared"),
("SQBage"," age squared"),
("SQBhogar_total"," hogar_total squared"),
("SQBedjefe"," edjefe squared"),
("SQBhogar_nin"," hogar_nin squared"),
("SQBovercrowding"," overcrowding squared"),
("SQBdependency"," dependency squared"),
("SQBmeaned"," meaned squared"),
("agesq"," Age squared"),]

description = pd.DataFrame(description, columns=['varname', 'description'])

In [0]:
description

,varname,description
0,v2a1,Monthly rent payment
1,hacdor,=1 Overcrowding by bedrooms
2,rooms,number of all rooms in the house
3,hacapo,=1 Overcrowding by rooms
4,v14a,=1 has toilet in the household
5,refrig,=1 if the household has refrigerator
6,v18q,owns a tablet
7,v18q1,number of tablets household owns
8,r4h1,Males younger than 12 years of age
9,r4h2,Males 12 years of age and older



1.3 Check Null Data
---



In [0]:
t = df.isnull().sum() # isnull 인 애들의 갯수 파악 
p = df.isnull().count() # isnull 인 열의 전체 개수 파악
t = df.isnull().sum().sort_values(ascending = False)
m = pd.concat([t, p], axis=1, keys=['total', 'percent']) # concat 을 시킬건데 axis1즉 열로 할거고 key 는 이렇다


(+) Pandas Indexing Practice
---



In [0]:
des.loc[des['var name'] == '***', 'description'] 
# des 데이터프레임에 적힌 'var name' 열의 *** 에 해당하는 row에 대한 'description' 열의 값을 불러오겠다

In [0]:
des[des['varname'] == '***']['description']

# des 중 des['varname'] 이 '***'인 sub set 을 뽑아서 거기의 ['description']열의 정보를 뽑겠다


1.4 Filling missing values
---



In [0]:
df[].nunique() # 이 열의 유니크한 값이 몇개나 있는지 보겠다

In [0]:
df[df['edf'] == 'yes']['edf'].count()
# edf 열이 'yes'인 subset에서 ['edf'] 열의 정보를 뽑아 갯수를 세겠다


In [0]:
df.loc[(df['edf'] != 'yes' ) & (df['edf'] ! = 'no'), 'edf'].nunique
# edf 가 yes 도 아니고 no 도 아닌 'edf' 열의 정보를 뽑아 거기서 유니크 한 값이 몇개인지 세겠다

In [0]:
df_train.loc[(df_train['edjefa'] == "yes") & (df_train['parentesco1'] == 1), "edjefa"] = df_train.loc[(df_train['edjefa'] == "yes") & (df_train['parentesco1'] == 1), "escolari"]
# yes 이고 1인 'edf'열의 값을 yes 이고 1인 'esc'의 값으로 바꾸겠다

In [0]:
df_train['edjef'] = np.max(df_train[['edjefa','edjefe']], axis=1)
# 'edj' 열의 값들을 eda 나 ede 의 열의 값 중 큰 값으로 하겠다


In [0]:
description.loc[description['varname'] == 'abastaguano', 'description']
# des 불러오기

In [0]:
df_train.loc[(df_train.v14a ==  1) & (df_train.sanitario1 ==  1) & (df_train.abastaguano == 0), "v14a"] = 0
# v14a 가 1이고 sani 가 1 이고 abas 가 0 인 애들의 v14 의 의 값을 0으로 만들어라


In [0]:
df['rez'].fillna(0, inplace=True)
# rez 열의 na 값들을 0으로 바꿔라 inplace 는 대부분의 경우 해주어야 한다

In [0]:
df['v18'].value_counts()
# v18 열의 값들에 대한 각 개수를 보여줘라

In [0]:
sns.kdeplot(df.loc[df['tipo'] == 1, 'v2'], label = 'mot')
# kdeplot 을 그릴건데 그 대사은 df tip 열의 값이 1 인 모든 행에서 'v2' 값을 뽑은 애들이며 label 은 mot 라고 적는다
sns.kdeplot(df.loc[df['tipo'] == 0, 'v2'], label = 'mot')
# 같은 kde plot 을 그릴건데 이번에 0 값이다
plt.xscale('log') # x축의 모든 값을 다 보여주지 않고 로그를 취해서 10^1 ^2 ^3 이런식으로만 보여준다


# 2. Feature Engineering


2.1 Object Featrure replacement
---



In [0]:
feat_obj = [col for col in df.columns if df[col].dtype == 'object']
# 열의 데이터타입이 오브젝트인 df의 모든 열들의 이름을 리스트 형태로 집어넣겠다

In [0]:
df['dependency'] = np.sqrt(df[sqbdep])
# df에 새로운 depencency 라는 열을 만드는데 df[sqb**] 열에 sqrt 를 취해서 만들겠다

In [0]:
df_train['dependency'] = df_train['dependency'].replace({np.inf: 0})
# df dep 열 안에서 inf 인 값들을 으로 바꿔주겠다
# 바꾸자마자원래 값을 없에도록 = 으로 새롭게 갱신하겠다

def replace_dependency(x):
    if x == 'yes':
        return 10
    elif x == 'no':
        return 0
    else:
        return x

df_train['dependency'] = df_train['dependency'].apply(replace_dependency).astype(float)

# dep 열안에 있는 cat 값들을 바꾸기 위해 apply 를 취할건데, replace dep 함수를 만들어서
# cat feat 값들이 나오면 저런식으로 값을 리턴하도록 한다. 데이터 타입은 float 으로한다

In [0]:
df_train['roof_waste_material'] = np.nan
# 새로운 열을 만드는데 nan 값으로 채워져있도록 한다

def fill_roof_exception(x):
    if (x['techozinc'] == 0) and (x['techoentrepiso'] == 0) and (x['techocane'] == 0) and (x['techootro'] == 0):
        return 1
    else:
        return 0
# tech 값들이 모두 0 이면 그 값에 1을 넣는다

    

In [0]:
df_train['roof_waste_material'] = df_train.apply(lambda x: fill_roof_exception(x), axis = 1)
# 왜 여기선 위와 다르게 함수를 바로 넣지 않고 lambda 를  사ㅏ용했을까.
# 같은 형식같음. 굳이 다르다면 여기는 x 의 값을 한번 더 들어가야해서 ? 



2.2 Extract Cat Feat
---



In [0]:
binary_cat_feat = [col for col in df.columns if df[col].value_counts().shape[0] == 2]
# value count() 로 나오는 df 에서 shape 0 즉 shape 의 row 에 해당하는 부분의 수(즉 행의 수)
# 가 2 인 모든 col 들의 이름을 리스트로 모으겠다. nunique()를 써도 같은 결과가 나온다


2.3 Make new feat using cont feat
---



In [0]:
# Cont features 를 찾기 위해 binary 제거 / object 제거 / Target 제거
continuous_features = [col for col in df_train.columns if col not in binary_cat_features]
continuous_features = [col for col in continuous_features if col not in features_object]
continuous_features = [col for col in continuous_features if col not in ['Id', 'Target', 'idhogar']] 
# 소거법 적용하기 if col not in ***

In [0]:
print('There are {} continuous features'.format(len(continuous_features)))
for col in continuous_features:
    print('{}: {}'.format(col, description.loc[description['varname'] == col, 'description'].values))
    
# 프린트 할건데 cont feat 의 모든  열에 대해서 각 열마다 description 알려주라    
    

In [0]:
df.drop('tam', axis = 1 , inplace=True)
# tam 이라는 열을 drop 해서 inplace 는 기본으로 조작 할때 쓴다


In [0]:
df_train = + - 
df_test

# train 에 했으면 test 에도 똑같이 해야한다. 모든 적용사항은 train 을 기준으로 따라간다

In [0]:
family_size_feat = ['a', 'b', 'c']
new_feat = []

for ccol in family_size_feat:
    new_col_name = '{} {}'.format('v2a1', col)
    new_feat.append(new_col_name)
    df_tr[new_col_name] = df_tr['v2a1'] / df_tr[col]
    df_te[new_col_name]
    
# 새로운 feat 을 만들건데 v2a1에 familysize관련 feat 을 모두 나눠주는 식으로 만든다. 만들어진 애들 이름은 ㄸ로 리스트로 저장해둔다    

In [0]:
for col in new_feats:
    df_train[col].replace([np.inf], np.nan, inplace=True)
    df_train[col].fillna(0, inplace=True)
    
    df_test[col].replace([np.inf], np.nan, inplace=True)
    df_test[col].fillna(0, inplace=True)

# 새로운 피쳐들에게 replace로 inf를 nan 으로 바꾸고 
# fillna 를 써서 nan 값을 0으로 바꾼다. 
# 애초에 inf 를 0으로 바꾸면 안됨?

In [0]:
# 위 두개 셀을 v2a1 말고도 room 이랑 bedroom에도 적용한다. v2a1 부분만 바꾼다

In [0]:
print(df_train.shape, df_test.shape)
# shape 가 family_size_feat의 3배만큼 늘어나 있어야 한다

In [0]:
# 같은식으로 v18q1, qmobilephone, rez_esc 에 대해서도 한다. 

In [0]:
# rez_age = df[rez] / df[age] 를 만든다


2.3.2 Rich Features
---



In [0]:
df['AAA_x_BBB'] = df['AAA'] * df['BBB']

In [0]:
# wall and roof
for col1 in ['epared1', 'epared2', 'epared3']:
    for col2 in ['etecho1', 'etecho2', 'etecho3']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]
        
# wall 의 상태와 roof 의 상태를 0, 1 값으로 나타낸 각 세 열들에 대해서 
# 이중 for 문으로 총 9번의 행동을 수행하는데
# new col name 은 new_A_x_B 꼴로 만들고
# Train 과 test 모두 a 열과 b 열의 곱을 통해서 만든다
# 둘다 0, 1 값이므로 둘다 1인 경우, 즉 예를 들어서
# roof bad wall bad 인 어떤 instance 는 epa1 ete1의 곱만 1이 될 것이고
# 나머지 8개의 열은 0이 될 것이다 

In [0]:
# 이런식으로 epa - wall, ete - roof, evi-floor 을 각각 수행해준다
# 이건 결국 그 둘의 값이 이 인스턴스의 상태에 해당하는 애들만 1로 남겨준다
# one hot 을 하나만 한거에서 2개를 한걸로 바꾼 느낌 
# 사과 가지고 있니? -> 사과랑 배 가지고 있니?

In [0]:
# 이번엔 electricity 와 energy -> energy, 
# toilet 과  rubbish disposal 을 other infra, 
# toilet 과 water provision 을 water 로 새로 만들고자함

In [0]:
print(df_train.shape, df_test.shape)
# 곱의 형태로 이루어지므로 이중포문 을 곱해서 몇 개의 새로운 feat 이 생겼는지를 알 수 있음.
# 이걸 통해 제대로 만들어졌는지 확인

In [0]:
df_train['electronics'] = df_train['computer'] * df_train['mobilephone'] * df_train['television'] * df_train['v18q'] * df_train['refrig']
df_test['electronics'] = df_test['computer'] * df_test['mobilephone'] * df_test['television'] * df_test['v18q'] * df_test['refrig']

df_train['no_appliances'] = df_train['refrig'] + df_train['computer'] + df_train['television'] + df_train['mobilephone']
df_test['no_appliances'] = df_test['refrig'] + df_test['computer'] + df_test['television'] + df_test['mobilephone']

# 이번엔 elect, appliance 있는지 여부 등을 구함. 일렉의 경우 저 3가지 모두 있는 애들만 1로 남김
# 노 어플라이언스는 저 것들 중 몇개가 있는지를 구함. 

In [0]:
# Mix wall material of roof, floor, wall 들에 대해서도 같은 식으로 진행해줌. 
# 이건 도저히 무슨 뭔 값들인지 모르겠다

데이터들을 조작하고 필요없는 애들 - 모든 값이 0인 열들을 없에줌

In [0]:
cols_with_only_one_value = []
for col in df_train.columns:
    if col == 'Target':
        continue
    if df_train[col].value_counts().shape[0] == 1 or df_test[col].value_counts().shape[0] == 1:
        print(col)
        cols_with_only_one_value.append(col)
        
df_train.drop(cols_with_only_one_value, axis=1, inplace=True)
df_test.drop(cols_with_only_one_value, axis=1, inplace=True)        
# 이제 하나의 값만 가진 col 들을 없에 줄건데, 원래 col 안에 0 과 1 두개의 값은 존재해야함. 근데 그 col 의 값의 종류가 1개
# 라는 건 거기엔 0뿐이라는 뜻. 그래서 그런 col 들을 모두 없에주고 뭘 없에 줬는지 적어둠. 

다 조작하고 나서 train 과 test 열의 갯수가 동일하면 서로 같은 열을 가지고 있는지 확인해야함

In [0]:
cols_train = np.array(sorted[col for col in df.columns if col != 'Target'])
cols_test = np.array(sorted[df.columns])

# col 이름들을 반환해줄건데 train 의 경우 'Target 을 제외한 나머지 col'을 반환해주고
# 그걸 알파벳 순으로 정렬해서 np array 형태로 반환하게함
# 테스트의 경우 target 열이 없으므로 그냥 그대로 


In [0]:
(cols_train == cols_test).sum() == len(cols_train)
# 이렇게해서 col train 과 col test 가 같은지를 확인가능함
# 좌변은 col train 과 test 가 같은 애들의 갯수. 왼쪽은 col train 의 갯수


2.4 Aggregation features
---

In [0]:
def max_min(x):
    return x.max() - x.min()

# x 의 값의 범위를 알려줌

tqdm - progress 를 알려주는 라이브러리 
 https://github.com/tqdm/tqdm

In [0]:
agg_tr =pd.DataFrame()

for item in tqdm(feats): # tqdm은 이중포문이던 뭐든 그냥 가장 바깥 포문에 넣어주면 되는듯
    for i, function in enu(['mean', 'std', ....]): # function 종류들
        group_tr = df[item].groupby(df['idhogar']).agg(function) 
        # idhogar로 item에 대해서 group 지은 다음에 idhogar 별로 agg function 을 써줌 
        if i == 6:
            new col = 
        else:
            new col = # max min 함수의 경우 정해진 이름이 없어서 ''형식이 아님. 그래서 직접 지정해줌
        agg_tr[new col] = group_tr    
        # 당연히 테스트 도 같은식으로 해줬어야함    

print('new aggregate train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))
# 크기와 feature 확인

In [0]:
aggr_list = ['a', 'b' ...]
for item in tqdm(aggr_list):
    for function in ['count', 'sum']:
# 같은 방식으로 

print('new aggregate train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))
# 크기와 feature 확인

In [0]:
# 나머지 애들도 같은 방식으로. 이번엔 위에 애들과는 대르게 mean std도 쓰고 count sum 도 써줌
# 아마도 여기 애들은 숫자이고 2번째 애들은 원핫, 첫째애들은 패밀리사이즈 피쳐라서 그런듯
# 원핫이 mean std 가 필요없는건 충분히 이해감
# 패밀리 사이즈가 count sum 이 필요없는건 어차피 카운트 썸이 중요한 피쳐가 아니여서인듯
# 나머지 피쳐는 .. 중요한가보지..?

In [0]:
agg_tr = agg_tr.reset_index() # index 없에기
agg_te = agg_te.reset_index()

tr_agg = pd.merge(df, agg, on='idhogar') # merge 를 할건데 idhogar 을 join key 로 삼겠다
te = pd.merge(df, agg, on='idhogar') # join key가 필수는 아닌데 있으면 교집합들은 넣어줘야함
# merge 는 왼쪽에서 오른쪽 으로 합치는 것임!
# concat 은 위에서 아래로 

tr_agg.fillna(0, inplace=True)

In [0]:
aggr_list = [...]


for lugar in ['lugar1', ...]: # 지역들
    group_train = df[[lugar, 'idhogar'] + aggr_list].groupby([lugar, 'idhogar']).sum().reset_index()
    # grouptr은 df 중 특정 지역과 idhogar, aggr 열들이 있는 subset에서 lugar과 idhogar기준으로 그룹을 나눈
    # 애들을 lugar, idhogar마다 더한 index가 없는 애들이다
    grouptr.columns = [lugar, 'idhogar'] + ['new_lugar_idhogar_'][2:] # 뒤에 2:는 첫 둘이 lugar, idhogar 이기 때문인듯? 굳이 이렇게 쓸 이유가 있나
    
    train_agg = pd.merge(df, groupdf_tr, on=[lugar, 'idhogar'])
    
    
# 이런식으로 lugar 마다 lugar과 idhogar로 groupby를해서 값을 만들어줌. 여기선 다른 애들은 필요가 없어서
# agg(func) 보단 바로 sum을 쓴듯
# 테스트도 같은 방식으로 진행

In [0]:
# 같은게 한번 더있는데 왜인지는 모르겠음

In [0]:
# 이번엔 다른 feature들에 대해서 위에 했던 방식으로 진행함. 

In [0]:
train = train_agg.query('parentesco1==1')
# household 의 head 만 점수 책정에 썼다고 해서 train_agg 에서 parentesco1 이 1 인애들만 query함
# train_agg로 만들어서 관리하던게 그 이유

In [0]:
submission = test[['Id']]
# 테스트할 Id 따로 만들어두기

# train test 에 필요없는 정보 없에기 

# corr 통해 positive negative top 20 뽑기 


# 3. Feature selection using shap

In [0]:
binary_cat_features = [col for col in train.columns if train[col].value_counts().shape[0] == 2]
object_features = ['edjefe', 'edjefa']

categorical_feats = binary_cat_features + object_features # 카테고리컬 애들만 남김

In [0]:
def evaluate_macroF1_lgb(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True)
# F1 score 을 계산하는데 multi class 이므로 average는 macro 로 해주어야됨. 
# 이슈에서 말하는 내용은 f1 score 을 계산하는데 reshape를 어떻게 해야하는지 임

In [0]:
y = train['Target']
train.drop(columns=['Target'], inplace=True)

In [0]:
def print_execution_time(start):
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print('*'*20, "Execution ended in {:0>2}h {:0>2}m {:05.2f}s".format(int(hours),int(minutes),seconds), '*'*20)
    
# divmod 숫자와 나눌 숫자를 입력하면 몫과 나머지 를 반환함
# 얼마나 걸렸는지 말해줌
    

In [0]:
def extract_good_features_using_shap_LGB(params, SEED):
    clf = lgb.LGBMClassifier(objective='multiclass',
                             random_state=1989,
                             max_depth=params['max_depth'], 
                             learning_rate=params['learning_rate'],  
                             silent=True, 
                             metric='multi_logloss',
                             n_jobs=-1, n_estimators=10000, 
                             class_weight='balanced',
                             colsample_bytree = params['colsample_bytree'], 
                             min_split_gain= params['min_split_gain'], 
                             bagging_freq = params['bagging_freq'],
                             min_child_weight=params['min_child_weight'],
                             num_leaves = params['num_leaves'], 
                             subsample = params['subsample'],
                             reg_alpha= params['reg_alpha'],
                             reg_lambda= params['reg_lambda'],
                             num_class=len(np.unique(y)),
                             bagging_seed=SEED,
                             seed=SEED,
                            )

    kfold = 5
    kf = StratifiedKFold(n_splits=kfold, shuffle=True)
    feat_importance_df  = pd.DataFrame()

    for i, (train_index, test_index) in enumerate(kf.split(train, y)):
        print('='*30, '{} of {} folds'.format(i+1, kfold), '='*30)
        start = time.time()
        X_train, X_val = train.iloc[train_index], train.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric=evaluate_macroF1_lgb, categorical_feature=categorical_feats,
                early_stopping_rounds=500, verbose=500)
        shap_values = shap.TreeExplainer(clf.booster_).shap_values(X_train)
        fold_importance_df  = pd.DataFrame()
        fold_importance_df['feature'] = X_train.columns
        fold_importance_df['shap_values'] = abs(np.array(shap_values)[:, :].mean(1).mean(0))
        fold_importance_df['feat_imp'] = clf.feature_importances_
        feat_importance_df = pd.concat([feat_importance_df, fold_importance_df])
        print_execution_time(start)

    feat_importance_df_shap = feat_importance_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()
#     feat_importance_df_shap['shap_cumsum'] = feat_importance_df_shap['shap_values'].cumsum() / feat_importance_df_shap['shap_values'].sum()
#     good_features = feat_importance_df_shap.loc[feat_importance_df_shap['shap_cumsum'] < 0.999].feature
    return feat_importance_df_shap

용어 정리

fit - early stopping: 오버언더 피팅 방지에서 나오는 그 얼리 스탑
fit - verbose: logging information 을 보여줌. 안하면 보통 속도가 매우 빨라짐

shap - https://shap.readthedocs.io/en/latest/# 참고



In [0]:
total_shap_df  = pd.DataFrame()
NUM_ITERATIONS = 50
for SEED in range(NUM_ITERATIONS):
    print('#'*40, '{} of {} iterations'.format(SEED+1, NUM_ITERATIONS), '#' * 40)
    params = {'max_depth': np.random.choice([5, 6, 7, 8, 10, 12, -1]),
             'learning_rate': np.random.rand() * 0.02,
              'colsample_bytree': np.random.rand() * (1 - 0.5) + 0.5,
              'subsample': np.random.rand() * (1 - 0.5) + 0.5,
              'min_split_gain': np.random.rand() * 0.2,
              'num_leaves': np.random.choice([32, 48, 64]),
              'reg_alpha': np.random.rand() * 2,
              'reg_lambda': np.random.rand() *2,
              'bagging_freq': np.random.randint(4) +1,
              'min_child_weight': np.random.randint(100) + 20
             }
    temp_shap_df = extract_good_features_using_shap_LGB(params, SEED)
    total_shap_df = pd.concat([total_shap_df, temp_shap_df])

In [0]:
shap_sorted_df = total_shap_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()
feat_imp_sorted_df = total_shap_df.groupby('feature').mean().sort_values('feat_imp', ascending=False).reset_index()
features_top_shap = shap_sorted_df['feature'][:500] # 중요한 feat 500개씩 
features_top_feat_imp = feat_imp_sorted_df['feature'][:500] # 여기서도 500개
top_features = pd.Series(features_top_shap.tolist() + features_top_feat_imp.tolist())
top_features = top_features.unique() # 중복된 feature는 없에줌
# Feature 에 따라 shap와 feat_imp 를 구분해서 가장 중요한 feat을 ㅊ찾음


# Model Development

In [0]:
new_tr = tr[top_features].copy()
# 중요한 feat만으로 새로 만들어줌

In [0]:
# extract_good_features_using_shap_LGB 에서 나온 feature importance 들을 위에서 뭐가 가장 중요할지등을 나타내었음
# 이제 그걸로 새로운 모델을 만들 거임
# new_tr 에 대해서 lgb 모델 만듬. 
# 이번 반환은 predict result 와 feat imp df

In [0]:
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
feat_importance_df_shap = feat_importance_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

num_features = 50
sns.barplot(x=feat_importance_df_shap.shap_values[:num_features], y=feat_importance_df_shap.feature[:num_features], ax=ax[0])
ax[0].set_title('Feature importance based on shap values')

feat_importance_df = feat_importance_df.groupby('feature').mean().sort_values('feat_imp', ascending=False).reset_index()

num_features = 50
sns.barplot(x=feat_importance_df.shap_values[:num_features], y=feat_importance_df.feature[:num_features], ax=ax[1])
ax[1].set_title('Feaure importance based on feature importance from lgbm')
plt.show()

# 결과 플로팅

In [0]:
submission['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission.to_csv('submission_with_new_feature_set.csv', index = False)


4.2 Random search
---



In [0]:
optimized_param = None
lowest_cv = 1000
total_iteration = 100
for i in range(total_iteration):
    print('-'*20, 'For {} of {} iterations'.format(i+1, total_iteration), '-'*20)
    learning_rate = np.random.rand() * 0.02
    n_folds = 3

    num_class = len(np.unique(y))

    params = {}
    params['application'] = 'multiclass'
    params['metric'] = 'multi_logloss'
    params['num_class'] = num_class
    params['class_weight'] = 'balanced'
    params['num_leaves'] = np.random.randint(24, 48)
    params['max_depth'] = np.random.randint(5, 8)
    params['min_child_weight'] = np.random.randint(5, 50)
    params['min_split_gain'] = np.random.rand() * 0.09
    params['colsample_bytree'] = np.random.rand() * (0.9 - 0.1) + 0.1
    params['subsample'] = np.random.rand() * (1 - 0.8) + 0.8
    params['bagging_freq'] = np.random.randint(1, 5)
    params['bagging_seed'] = np.random.randint(1, 5)
    params['reg_alpha'] = np.random.rand() * 2
    params['reg_lambda'] = np.random.rand() * 2
    params['learning_rate'] = np.random.rand() * 0.02
    params['seed']  =1989

    d_train = lgb.Dataset(data=new_train, label=y.values-1, categorical_feature=new_categorical_feats, free_raw_data=False)
    cv_results = lgb.cv(params=params, train_set=d_train, num_boost_round=10000, categorical_feature=new_categorical_feats,
                        nfold=n_folds, stratified=True, shuffle=True, early_stopping_rounds=1, verbose_eval=1000)

    min_cv_results = min(cv_results['multi_logloss-mean'])

    if min_cv_results < lowest_cv:
        lowest_cv = min_cv_results
        optimized_param = params